## 特征提取

In [1]:
import sys, platform, os, time
import mindspore as ms
from mindspore import context

[WARNING] DEVICE(8007,e7ffc1dc5020,python):2025-08-15-18:25:46.861.807 [mindspore/ccsrc/utils/dlopen_macro.h:165] DlsymAscend] Dynamically load symbol aclprofGetSupportedFeaturesV2 failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libmsprofiler.so: undefined symbol: aclprofGetSupportedFeaturesV2
[WARNING] DEVICE(8007,e7ffc1dc5020,python):2025-08-15-18:25:46.861.936 [mindspore/ccsrc/utils/dlopen_macro.h:165] DlsymAscend] Dynamically load symbol aclrtEventGetTimestamp failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libascendcl.so: undefined symbol: aclrtEventGetTimestamp
/usr/local/miniconda3/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/miniconda3/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  re

依赖检查

In [2]:
import importlib, subprocess, sys

def ensure(pkg, ver=None):
    try:
        m = importlib.import_module(pkg)
        if ver:
            print(f"{pkg}=={getattr(m,'__version__','?')} (要求 {ver})")
        else:
            print(f"{pkg} 已安装")
    except Exception as e:
        print(f"缺失 {pkg}，开始安装……")
        cmd = [sys.executable, "-m", "pip", "install", pkg + (f"=={ver}" if ver else "")]
        print(">>>", " ".join(cmd))
        subprocess.check_call(cmd)

ensure("mindnlp", "0.4.1")
ensure("numpy")
ensure("scikit_learn")

[WARNING] ME(8007:255085655117856,MainProcess):2025-08-15-18:25:53.460.708 [mindspore/context.py:1402] For 'context.set_context', the parameter 'ascend_config' will be deprecated and removed in a future version. Please use the api mindspore.device_context.ascend.op_precision.precision_mode(),
                                                       mindspore.device_context.ascend.op_precision.op_precision_mode(),
                                                       mindspore.device_context.ascend.op_precision.matmul_allow_hf32(),
                                                       mindspore.device_context.ascend.op_precision.conv_allow_hf32(),
                                                       mindspore.device_context.ascend.op_tuning.op_compile() instead.
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 2.137 seconds.
Prefix dict has been built successfully.


mindnlp==? (要求 0.4.1)
numpy 已安装
缺失 scikit_learn，开始安装……
>>> /usr/local/miniconda3/bin/python -m pip install scikit_learn
Defaulting to user installation because normal site-packages is not writeable


从 Hugging Face 官方源加载 BGE small zh

In [3]:
from mindnlp.transformers import AutoTokenizer, AutoModel

MODEL_ID = "BAAI/bge-small-zh-v1.5"  
print("Loading:", MODEL_ID)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)   
model = AutoModel.from_pretrained(MODEL_ID)
model.set_train(False)

Loading: BAAI/bge-small-zh-v1.5


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 512, padding_idx=0)
    (position_embeddings): Embedding(512, 512)
    (token_type_embeddings): Embedding(2, 512)
    (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-3): 4 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear (512 -> 512)
            (key): Linear (512 -> 512)
            (value): Linear (512 -> 512)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear (512 -> 512)
            (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Linear (512 -> 2048)
          (intermediate_act_fn):

文本特征提取

In [4]:
import numpy as np
import mindspore.ops as ops

def encode_texts(texts, max_len=256, batch_size=16):
    all_vecs = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        enc = tokenizer(batch, padding=True, truncation=True, max_length=max_len, return_tensors="ms")
        out = model(**enc)
        mask = enc["attention_mask"].astype(ms.float32).expand_dims(-1) 
        summed = ops.sum(out.last_hidden_state * mask, dim=1, keepdim=False)  
        counts = ops.clip_by_value(ops.sum(mask, dim=1, keepdim=False), 1.0, 1e9)
        mean_pooled = summed / counts  
        all_vecs.append(mean_pooled.asnumpy())
    return np.vstack(all_vecs)

In [6]:
texts = [
    "生活就像一杯茶，慢慢品味才能感受其中的香醇。",
    "科学技术是第一生产力。",
    "读书破万卷，下笔如有神。"
]
embeddings = encode_texts(texts)
print("向量形状:", embeddings.shape)  
for idx, sent in enumerate(texts):
    print(f"句子 {idx+1}：{sent}")
    print("前5维示例:", np.round(embeddings[idx][:5], 6))
    print()

向量形状: (3, 512)
句子 1：生活就像一杯茶，慢慢品味才能感受其中的香醇。
前5维示例: [ 0.286835 -0.009755  0.012051 -0.090807 -0.450408]

句子 2：科学技术是第一生产力。
前5维示例: [ 0.071481  0.325284  0.453272 -0.222929 -0.320808]

句子 3：读书破万卷，下笔如有神。
前5维示例: [-0.186404  0.551733  0.001602 -0.302225 -0.129176]

